In [8]:
import pandas as pd
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import mysql.connector

In [32]:
# 데이터 확인

data = pd.read_excel("../data/transport_time_2016.xlsx", sheet_name=1, usecols=(0, 5, 6, 7, 8, 9, 11, 14))
data = data[data["Time_Zone"] == "00_AllDay"]
data.tail()

,Year,Time_Zone,Mode,HDCD_SD_NM,HDCD_SGG_NM,HDCD_EMD_NM,Faci_NM,평균접근시간(분)
109185,2016,00_AllDay,02_PT,경기도,양평군,개군면,광역교통-버스터미널,89.537256
109192,2016,00_AllDay,01_Car,경기도,양평군,개군면,광역교통-철도역,18.116527
109193,2016,00_AllDay,02_PT,경기도,양평군,개군면,광역교통-철도역,94.920071
109200,2016,00_AllDay,01_Car,경기도,양평군,개군면,광역교통-공항,119.603274
109201,2016,00_AllDay,02_PT,경기도,양평군,개군면,광역교통-공항,201.610483


In [49]:
data = data[data["Time_Zone"] == "00_AllDay"]
data = data.drop(columns="Time_Zone")
# data_PT = data[data["Mode"].str.contains(mode)]
# data = data_PT.drop(columns=["Time_Zone", "Mode"])
# mode = "대중교통"

# type 내용 통합
data = data.replace(to_replace = education_list, value= "교육")
data = data.replace(to_replace = medical_list, value= "의료")
data = data.replace(to_replace = commercial_list, value= "판매")
data = data.replace(to_replace = transport_list, value= "교통")
data = data.replace(to_replace= "-", value= "전체")

# 칼럼 이름 변경
data.columns = ["YEAR", "Mode", "SIDO", "GUNGU", "DONG", "TYPE", "AVG_ACC_MINUTE"]

# 피봇테이블로 type 별 평균 구하기
df = pd.pivot_table(data,
                    index= ["YEAR", "Mode", "SIDO", "GUNGU", "DONG"],
                    columns= ["TYPE"],
                    values= "AVG_ACC_MINUTE")
#df.reset_index(inplace=True)
df.head()

KeyError: 'Time_Zone'

In [50]:
df = pd.pivot_table(data,
                    index= ["YEAR", "Mode", "SIDO", "GUNGU", "DONG"],
                    columns= ["TYPE"],
                    values= "AVG_ACC_MINUTE")
df.tail()

TYPE                                교육         교통         의료         판매
YEAR Mode  SIDO  GUNGU DONG                                            
2016 02_PT 인천광역시 중구    영종동   15.925227  42.339191  25.296488  39.815661
                       용유동   25.732204  49.382155  36.174672  59.528563
                       운서동    6.115531  33.617458  24.278923  29.157598
                       율목동    4.701375  42.395652   5.918180   6.516764
                       전체     9.097366  42.266622  17.829295  23.830436

In [56]:
df.head()

TYPE                                교육         교통         의료         판매
YEAR Mode   SIDO GUNGU DONG                                            
2016 01_Car 경기도  가평군   가평읍    4.786721  43.091057  26.732271  27.860993
                       북면    16.170148  66.827810  40.911471  50.782702
                       상면    11.685859  52.371382  25.201252  40.228015
                       설악면    5.168481  56.958842  21.736761  41.185691
                       전체     6.036235  48.424847  24.047643  34.291849

In [57]:
df.reset_index(inplace=True)

In [60]:
pivot = df.melt(id_vars=["YEAR", "Mode", "SIDO", "GUNGU", "DONG"])
pivot

,YEAR,Mode,SIDO,GUNGU,DONG,TYPE,value
0,2016,01_Car,경기도,가평군,가평읍,교육,4.786721
1,2016,01_Car,경기도,가평군,북면,교육,16.170148
2,2016,01_Car,경기도,가평군,상면,교육,11.685859
3,2016,01_Car,경기도,가평군,설악면,교육,5.168481
4,2016,01_Car,경기도,가평군,전체,교육,6.036235
...,...,...,...,...,...,...,...
9683,2016,02_PT,인천광역시,중구,영종동,판매,39.815661
9684,2016,02_PT,인천광역시,중구,용유동,판매,59.528563
9685,2016,02_PT,인천광역시,중구,운서동,판매,29.157598
9686,2016,02_PT,인천광역시,중구,율목동,판매,6.516764


In [61]:
pivot = pivot.replace(to_replace= "01_Car", value= "자가용")
pivot = pivot.replace(to_replace= "01_PT", value= "대중교통")

In [63]:
# 쿼리

sql = "INSERT INTO TRANSPORTATION_ACCESSIBILITY \
    (YEAR, MODE, SIDO, GUNGU, DONG, CATEGORY, AVG_MINUTE) \
        VALUES (%s, %s, %s, %s, %s, %s, %s)"

In [65]:
for i, row in tqdm(pivot.iterrows(), total= pivot.shape[0]):
    cur.execute(sql, tuple(row))
#    print(tuple(row))
    remote.commit()

100%|██████████| 9688/9688 [03:12<00:00, 50.32it/s]


In [53]:
df.stack()

YEAR  Mode    SIDO   GUNGU  DONG  TYPE
2016  01_Car  경기도    가평군    가평읍   교육       4.786721
                                  교통      43.091057
                                  의료      26.732271
                                  판매      27.860993
                            북면    교육      16.170148
                                            ...    
      02_PT   인천광역시  중구     율목동   판매       6.516764
                            전체    교육       9.097366
                                  교통      42.266622
                                  의료      17.829295
                                  판매      23.830436
Length: 9688, dtype: float64

In [6]:
# 연도별 파일 전부 불러오기

file_list = glob("../data/transport*")
print(len(file_list))
file_list

7


['../data/transport_time_2020.xlsx',
 '../data/transport_time_2019.xlsx',
 '../data/transport',
 '../data/transport_time_2016.xlsx',
 '../data/transport_time_2017.xlsx',
 '../data/transport_time_2018.xlsx',
 '../data/transport_time_2021.xlsx']

In [12]:
file_list.pop(2)

'../data/transport'

In [21]:
file_list.pop(2)

'../data/transport_time_2016.xlsx'

In [24]:
file_list

['../data/transport_time_2020.xlsx',
 '../data/transport_time_2019.xlsx',
 '../data/transport_time_2017.xlsx',
 '../data/transport_time_2018.xlsx',
 '../data/transport_time_2021.xlsx']

In [23]:
data = pd.read_excel(file_list[3], usecols=(0, 5, 6, 7, 8, 9, 11, 14), header=4)
data.tail()

,Year,Time_Zone,Mode,HDCD_SD_NM,HDCD_SGG_NM,HDCD_EMD_NM,Faci_CA,Mode_NM
348211,2018,1_T0709,2_PT,제주특별자치도,서귀포시,예래동,광역교통시설,대중교통/도보
348212,2018,2_T1214,1_PC,제주특별자치도,서귀포시,예래동,광역교통시설,승용차
348213,2018,2_T1214,2_PT,제주특별자치도,서귀포시,예래동,광역교통시설,대중교통/도보
348214,2018,3_T1820,1_PC,제주특별자치도,서귀포시,예래동,광역교통시설,승용차
348215,2018,3_T1820,2_PT,제주특별자치도,서귀포시,예래동,광역교통시설,대중교통/도보


# DB 연결

In [66]:
# 쿼리

sql = "INSERT INTO TRANSPORTATION_ACCESSIBILITY \
    (YEAR, MODE, SIDO, GUNGU, DONG, CATEGORY, AVG_MINUTE) \
        VALUES (%s, %s, %s, %s, %s, %s, %s)"

In [9]:
education_list = ['교육-초등학교', '교육-중학교', '교육-고등학교']
medical_list = ['의료-공공의료시설', '의료-병·의원', '의료-종합병원']
commercial_list = ['판매-대규모점포', '판매-전통시장',]
transport_list = ['광역교통-버스터미널', '광역교통-철도역', '광역교통-공항']

In [92]:
def change_mode(data):
    data = data.replace(to_replace= "1_PC", value= "자가용")
    data = data.replace(to_replace= "2_PT", value= "대중교통")
    return data

def merge_type(data):
    data = data.replace(to_replace = education_list, value= "교육")
    data = data.replace(to_replace = medical_list, value= "의료")
    data = data.replace(to_replace = commercial_list, value= "판매")
    data = data.replace(to_replace = transport_list, value= "교통")
    return data

def rename_column(data):
    data.columns = ["YEAR", "SIDO", "GUNGU", "DONG", "TYPE", "AVG_ACC_MINUTE"] 
    return data   

def casting_time(data):
    data["AVG_ACC_MINUTE"].replace(">120", 120.00, inplace=True)
    data["AVG_ACC_MINUTE"].replace(">120.00", 120.00, inplace=True)
    data["AVG_ACC_MINUTE"].astype(float)
    return data

In [98]:
for file in file_list:
# 파일 불러오기 - 일 평균값, 대중교통만 사용
    data = pd.read_excel(file, usecols=(0, 5, 6, 7, 8, 9, 11, 15), header=4).fillna(0)
#    data = data[data["Time_Zone"] == "00_AllDay"]
    data = data[data["Time_Zone"] == "0_AllDay"]
    data = data.drop(columns="Time_Zone")

# mode 값 변경
    data = change_mode(data)

# 칼럼 이름 변경
    data.columns = ["YEAR", "MODE", "SIDO", "GUNGU", "DONG", "TYPE", "AVG_ACC_MINUTE"]

# 120분 이상 값 숫자형식으로 변경
    data = casting_time(data)

# 피봇테이블로 type 별 평균 구하기
    df = pd.pivot_table(data,
                        index= ["YEAR", "MODE", "SIDO", "GUNGU", "DONG"],
                        columns= ["TYPE"],
                        values= "AVG_ACC_MINUTE")
    df.reset_index(inplace=True)
    df_melt = df.melt(id_vars=["YEAR", "MODE", "SIDO", "GUNGU", "DONG"])
    df_melt
    
    for i, row in tqdm(df_melt.iterrows(), total= df_melt.shape[0]):
        cur.execute(sql, tuple(row))
        remote.commit()

    print(file, "끝!!!!!")

/tmp/ipykernel_135662/3920219560.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["AVG_ACC_MINUTE"].replace(">120", 120.00, inplace=True)
/tmp/ipykernel_135662/3920219560.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, in

../data/transport_time_2020.xlsx 끝!!!!!


/tmp/ipykernel_135662/3920219560.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["AVG_ACC_MINUTE"].replace(">120", 120.00, inplace=True)
/tmp/ipykernel_135662/3920219560.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, in

../data/transport_time_2019.xlsx 끝!!!!!


/tmp/ipykernel_135662/3920219560.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["AVG_ACC_MINUTE"].replace(">120", 120.00, inplace=True)
/tmp/ipykernel_135662/3920219560.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, in

../data/transport_time_2018.xlsx 끝!!!!!


KeyError: 'Time_Zone'

In [95]:
file_list[1]

'../data/transport_time_2019.xlsx'

In [96]:
file_list

['../data/transport_time_2020.xlsx',
 '../data/transport_time_2019.xlsx',
 '../data/transport_time_2017.xlsx',
 '../data/transport_time_2018.xlsx',
 '../data/transport_time_2021.xlsx']

In [97]:
file_list.pop(2)

'../data/transport_time_2017.xlsx'

In [81]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 86768 entries, 0 to 347065
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   YEAR            86768 non-null  int64 
 1   MODE            86768 non-null  object
 2   SIDO            86768 non-null  object
 3   GUNGU           86768 non-null  object
 4   DONG            86768 non-null  object
 5   TYPE            86768 non-null  object
 6   AVG_ACC_MINUTE  86768 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [89]:
data["AVG_ACC_MINUTE"].replace(">120", 120.00, inplace=True)
data["AVG_ACC_MINUTE"].replace(">120.00", 120.00, inplace=True)

/tmp/ipykernel_135662/3474083213.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data["AVG_ACC_MINUTE"].replace(">120", 120.00, inplace=True)


In [90]:
data["AVG_ACC_MINUTE"].astype(float)

0           3.099632
1           6.990393
8           4.338199
9          10.541317
16          5.848187
             ...    
347049     38.262778
347056    120.000000
347057    120.000000
347064     64.784548
347065     76.915681
Name: AVG_ACC_MINUTE, Length: 86768, dtype: float64

In [91]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 86768 entries, 0 to 347065
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   YEAR            86768 non-null  int64  
 1   MODE            86768 non-null  object 
 2   SIDO            86768 non-null  object 
 3   GUNGU           86768 non-null  object 
 4   DONG            86768 non-null  object 
 5   TYPE            86768 non-null  object 
 6   AVG_ACC_MINUTE  86768 non-null  float64
dtypes: float64(1), int64(1), object(5)
memory usage: 5.3+ MB


In [100]:
file_list[3]

'../data/transport_time_2021.xlsx'

In [103]:
# 파일 불러오기 - 일 평균값, 대중교통만 사용
data = pd.read_excel(file_list[3],sheet_name=1, usecols=(0, 5, 6, 7, 8, 9, 11, 15), header=4).fillna(0)
data.head()

,Year,Time_Zone,Mode,HDCD_SD_NM,HDCD_SGG_NM,HDCD_EMD_NM,Faci_CA,평균접근시간(분)
0,2021,0_AllDay,1_PC,전국,-,-,교육시설,2.262503
1,2021,0_AllDay,2_PT,전국,-,-,교육시설,6.941821
2,2021,1_T0709,1_PC,전국,-,-,교육시설,2.154142
3,2021,1_T0709,2_PT,전국,-,-,교육시설,6.807373
4,2021,2_T1214,1_PC,전국,-,-,교육시설,2.251829


In [104]:
data = data[data["Time_Zone"] == "0_AllDay"]
data = data.drop(columns="Time_Zone")

# mode 값 변경
data = change_mode(data)

# 칼럼 이름 변경
data.columns = ["YEAR", "MODE", "SIDO", "GUNGU", "DONG", "TYPE", "AVG_ACC_MINUTE"]

# 120분 이상 값 숫자형식으로 변경
data = casting_time(data)

# 피봇테이블로 type 별 평균 구하기
df = pd.pivot_table(data,
                    index= ["YEAR", "MODE", "SIDO", "GUNGU", "DONG"],
                    columns= ["TYPE"],
                    values= "AVG_ACC_MINUTE")
df.reset_index(inplace=True)
df_melt = df.melt(id_vars=["YEAR", "MODE", "SIDO", "GUNGU", "DONG"])
df_melt

for i, row in tqdm(df_melt.iterrows(), total= df_melt.shape[0]):
    cur.execute(sql, tuple(row))
    remote.commit()

print(file, "끝!!!!!")

/tmp/ipykernel_135662/3920219560.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["AVG_ACC_MINUTE"].replace(">120", 120.00, inplace=True)
/tmp/ipykernel_135662/3920219560.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, in

../data/transport_time_2021.xlsx 끝!!!!!


# 서버 닫기

In [105]:
remote.close()